In [1]:
using CSV, DataFrames, JuMP, Gurobi, LinearAlgebra, Random

In [2]:
# Constants

nP = 10 # number of people
nT = 100 # number of timeslots in a week
cM = 1e7 # big number

print("")

In [3]:
# Load Data

A = zeros(nP, nT) # 0-1 matrix of whether person can mentor/mentee at time T

# All of below are vectors of size n

aR = zeros(nP, 1) # max num of session person can mentor
aE = zeros(nP, 1) # max num of session person can mentee
r  = zeros(nP, 1) # rank
pR = zeros(nP, 1) # 0-1 does mentor prefer poomsae 
pE = zeros(nP, 1) # 0-1 does mentee prefer poomsae
sR = zeros(nP, 1) # 0-1 does mentor prefer sparring
sE = zeros(nP, 1) # 0-1 does mentee prefer sparring

print("")

In [4]:
# Create Cost Matrix

alpha = 1
beta  = 1

C = zeros(nP, nP)

for i = 1:nP, j = 1:nP
    C[i, j] += alpha * (r[i]-r[j]-2+(r[j] >= 9))
    C[i, j] -= beta  * (pR[i]*pE[j]+sR[i]*sE[j]-pR[i]*sR[i]-pE[j]*sE[j])
    C[i, j] -= 1
end

In [5]:
# Rank Constraint Matrix

R = zeros(nP, nP, nT)

for i = 1:nP, j = 1:nP, T = 1:nT
    if r[i] >= r[j]+2 || (r[i] >= r[j]+1 && r[j] >= 9)
        R[i, j, T] = 1
    end
end

In [6]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

print("")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-23


In [7]:
@variable(m, 0 <= x[1:nP, 1:nP, 1:nT] <= 1)
@variable(m, 0 <= t[1:nP] <= 1, Int)

print("")

In [8]:
# Rank Constraint
@constraint(m, x .<= R)

# Maximum Constraints
for P = 1:nP
    @constraint(m, sum(x[i, P, T] for i=1:nP, T = 1:nT) <= aR[P])
    @constraint(m, sum(x[P, i, T] for i=1:nP, T = 1:nT) <= aE[P])
end

# Availability Constraints
for i = 1:nP, j = i+1:nP, T=1:nT
    @constraint(m, x[i,j,T] <= A[i,T]*A[j,T])
end

# Single Mentor Constraint
for i = 1:nP, j = i+1:nP, k = 1:nP, T=1:nT
    @constraint(m, x[i,k,T]+x[j,k,T] <= 1)
end

# Making t work
for P = 1:nP
    @constraint(m, sum(x[i,P,T] for i=1:nP, T=1:nT) >= t[P])
    @constraint(m, 0.01*sum(x[i,P,T] for i=1:nP, T=1:nT) <= t[P])
end

print("")

In [9]:
@objective(m, Min, cM*sum((1-t[P]) for P=1:nP) + sum(x[i,j,T]*C[i,j] for i=1:nP, j=1:nP, T=1:nT))
print("")